In [11]:
# Авто-установка OpenCV при необходимости
import importlib, sys, subprocess

try:
    importlib.import_module("cv2")
except ModuleNotFoundError:
    print("Installing opencv-python-headless...")
    subprocess.check_call([sys.executable, "-m", "pip", "install", "--quiet", "opencv-python-headless"])
    importlib.invalidate_caches()
    import cv2  # noqa


In [12]:
# Импорт библиотек
import os
import glob
import csv
from dataclasses import dataclass
from typing import List, Tuple

import numpy as np
import cv2



In [14]:
# Конфигурация путей и параметров (только улучшение качества)
IMAGES_DIR = "/Users/sosiska_killer/Documents/diplom/research_algorithm/Vega_detect/images"
OUTPUT_DIR = "/Users/sosiska_killer/Documents/diplom/research_algorithm/Vega_detect/output/auto_enhanced"
INTER_DIR = os.path.join(OUTPUT_DIR, "intermediate")
ENH_DIR = os.path.join(OUTPUT_DIR, "enhanced")
for d in [OUTPUT_DIR, INTER_DIR, ENH_DIR]:
    os.makedirs(d, exist_ok=True)

# Параметры фильтрации/улучшения
WINDOW_SIZE = 7            # размер окна для Lee
LEE_DAMPING_K = 1.0        # коэффициент подавления шума (0.5-2.0)

# Параметры контраста (затемнение темных + осветление светлых)
GAMMA_DARK = 0.7           # затемнение темных участков (0.5-1.0, меньше = темнее)
GAMMA_LIGHT = 1.3          # осветление светлых участков (1.0-2.0, больше = светлее)
THRESHOLD = 0.5            # порог разделения темных/светлых (0-1)

# Список входных файлов (все форматы изображений)
image_extensions = ['*.jpg', '*.jpeg', '*.png', '*.bmp', '*.tiff', '*.tif', '*.gif', '*.webp', '*.tga', '*.dds', '*.exr', '*.hdr', '*.ppm', '*.pgm', '*.pbm']
image_paths = []
for ext in image_extensions:
    image_paths.extend(glob.glob(os.path.join(IMAGES_DIR, ext)))
    image_paths.extend(glob.glob(os.path.join(IMAGES_DIR, ext.upper())))  # заглавные расширения

image_paths = sorted(list(set(image_paths)))  # убираем дубликаты и сортируем
print(f"Найдено изображений: {len(image_paths)}")
if image_paths:
    print(f"Форматы: {set([os.path.splitext(p)[1].lower() for p in image_paths])}")
    print(f"Примеры: {[os.path.basename(p) for p in image_paths[:3]]}")
else:
    print("Изображения не найдены!")


Найдено изображений: 9
Форматы: {'.jpg'}
Примеры: ['1.jpg', '2.jpg', '3.jpg']


In [15]:
# Функции: нормализация и Lee-фильтр

def normalize_to_uint8(img: np.ndarray) -> np.ndarray:
    if img.dtype != np.float32 and img.dtype != np.float64:
        img = img.astype(np.float32)
    mn, mx = np.min(img), np.max(img)
    if mx - mn < 1e-9:
        return np.zeros_like(img, dtype=np.uint8)
    out = (img - mn) / (mx - mn)
    out = (out * 255.0).clip(0, 255).astype(np.uint8)
    return out


def lee_filter(image: np.ndarray, window_size: int = 7, k: float = 1.0) -> np.ndarray:
    """Классический Lee-фильтр (интенсивность).
    image: ожидается uint8 или float32, 1 канал.
    window_size: нечетное число.
    k: коэффициент подавления шума (0.5..2.0)
    """
    if image.ndim == 3:
        image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    if image.dtype != np.float32:
        img = image.astype(np.float32)
    else:
        img = image

    half = window_size // 2
    # Скользящие среднее и дисперсия через боковой фильтр
    mean = cv2.boxFilter(img, ddepth=-1, ksize=(window_size, window_size), normalize=True, borderType=cv2.BORDER_REFLECT)
    mean_sq = cv2.boxFilter(img * img, ddepth=-1, ksize=(window_size, window_size), normalize=True, borderType=cv2.BORDER_REFLECT)
    var = (mean_sq - mean * mean).clip(0)

    # Оценка дисперсии шума: медиана по var
    noise_var = np.median(var)
    # Вес по Ли
    w = var / (var + k * noise_var + 1e-12)
    out = mean + w * (img - mean)
    return out.astype(np.float32)



In [16]:
# Функции: улучшение контраста (затемнение темных + осветление светлых)

def enhance_contrast_dark_light(img_f32: np.ndarray,
                                gamma_dark: float = 0.7,      # затемнение темных (0.5-1.0)
                                gamma_light: float = 1.3,     # осветление светлых (1.0-2.0)
                                threshold: float = 0.5) -> np.ndarray:
    """Улучшение контраста: затемняет темные участки и осветляет светлые.
    threshold: порог разделения темных/светлых участков (0-1)
    """
    # Нормализуем к [0,1]
    img_norm = normalize_to_uint8(img_f32).astype(np.float32) / 255.0
    
    # Создаем маски для темных и светлых участков
    dark_mask = img_norm < threshold
    light_mask = img_norm >= threshold
    
    # Применяем гамма-коррекцию отдельно к темным и светлым участкам
    result = img_norm.copy()
    
    # Затемняем темные участки
    if np.any(dark_mask):
        result[dark_mask] = np.power(img_norm[dark_mask], gamma_dark)
    
    # Осветляем светлые участки  
    if np.any(light_mask):
        result[light_mask] = np.power(img_norm[light_mask], gamma_light)
    
    # Возвращаем к uint8
    return (result * 255.0).clip(0, 255).astype(np.uint8)



In [17]:
# Функции: детекция малых объектов vs шум

def detect_small_objects(img_u8: np.ndarray,
                         min_area: int = 8,
                         max_area: int = 400,
                         morph_open: int = 3,
                         morph_close: int = 5) -> Tuple[np.ndarray, List[Tuple[int,int,int,int]]]:
    """Возвращает бинарную маску объектов и bounding-box'ы найденных малых объектов.
    Шум подавляется морфологией и порогом площади.
    """
    # Адаптивный порог (Otsu + небольшое усиление)
    _, th = cv2.threshold(img_u8, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)

    # Немного ужесточим порог (повышаем пороговую границу на 10%)
    hist_thr = int(max(0, min(255, 1.1 * _)))
    _, th2 = cv2.threshold(img_u8, hist_thr, 255, cv2.THRESH_BINARY)

    # Морфологическая фильтрация
    if morph_open > 0:
        k1 = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (morph_open, morph_open))
        th2 = cv2.morphologyEx(th2, cv2.MORPH_OPEN, k1)
    if morph_close > 0:
        k2 = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (morph_close, morph_close))
        th2 = cv2.morphologyEx(th2, cv2.MORPH_CLOSE, k2)

    # Поиск контуров и фильтрация по площади
    cnts, _h = cv2.findContours(th2, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    mask = np.zeros_like(th2)
    boxes: List[Tuple[int,int,int,int]] = []
    for c in cnts:
        area = cv2.contourArea(c)
        if area < min_area:
            # шум
            continue
        if area <= max_area:
            x,y,w,h = cv2.boundingRect(c)
            boxes.append((x,y,w,h))
            cv2.drawContours(mask, [c], -1, 255, thickness=-1)
        else:
            # более крупные объекты можно игнорировать или тоже отмечать
            pass
    return mask, boxes



In [18]:
# Пакетная обработка изображений: только улучшение качества

processed_count = 0
for idx, img_path in enumerate(image_paths, start=1):
    base = os.path.basename(img_path)
    name, ext = os.path.splitext(base)
    
    # Читаем изображение (поддерживает все форматы)
    img_bgr = cv2.imread(img_path, cv2.IMREAD_COLOR)
    if img_bgr is None:
        print(f"[WARN] Не удалось прочитать {img_path}")
        continue
    
    print(f"Обрабатываем {idx}/{len(image_paths)}: {base}")
    gray = cv2.cvtColor(img_bgr, cv2.COLOR_BGR2GRAY)

    # 1) Подавление спекл-шума (Lee)
    lee = lee_filter(gray, window_size=WINDOW_SIZE, k=LEE_DAMPING_K)

    # 2) Улучшение контраста (затемнение темных + осветление светлых)
    enhanced_u8 = enhance_contrast_dark_light(lee, gamma_dark=GAMMA_DARK, gamma_light=GAMMA_LIGHT, threshold=THRESHOLD)

    # 3) Сохранение в том же формате что и исходный файл
    inter_path = os.path.join(INTER_DIR, f"{name}_lee{ext}")
    enh_path = os.path.join(ENH_DIR, f"{name}_enhanced{ext}")
    
    # Определяем параметры сжатия в зависимости от формата
    if ext.lower() in ['.jpg', '.jpeg']:
        cv2.imwrite(inter_path, normalize_to_uint8(lee), [cv2.IMWRITE_JPEG_QUALITY, 95])
        cv2.imwrite(enh_path, enhanced_u8, [cv2.IMWRITE_JPEG_QUALITY, 95])
    elif ext.lower() in ['.png']:
        cv2.imwrite(inter_path, normalize_to_uint8(lee), [cv2.IMWRITE_PNG_COMPRESSION, 3])
        cv2.imwrite(enh_path, enhanced_u8, [cv2.IMWRITE_PNG_COMPRESSION, 3])
    else:
        # Для остальных форматов без дополнительных параметров
        cv2.imwrite(inter_path, normalize_to_uint8(lee))
        cv2.imwrite(enh_path, enhanced_u8)
    
    processed_count += 1

print(f"Готово. Успешно обработано файлов: {processed_count} из {len(image_paths)}")


Обрабатываем 1/9: 1.jpg
Обрабатываем 2/9: 2.jpg
Обрабатываем 3/9: 3.jpg
Обрабатываем 4/9: 4.jpg
Обрабатываем 5/9: 5.jpg
Обрабатываем 6/9: 6.jpg
Обрабатываем 7/9: 7.jpg
Обрабатываем 8/9: 8.jpg
Обрабатываем 9/9: 9.jpg
Готово. Успешно обработано файлов: 9 из 9
